$$\textrm{Joaquin Peñuela Parra, Cristian Fernando Rodriguez Cruz}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

In [1]:
import os
from itertools import product
import pandas as pd
import multiprocessing as mp

In [2]:
import subprocess

## IMPORTANT: Make sure that "Uniandes_Framework" is in .gitignore
framework_path = "Uniandes_Framework"

if os.path.exists(framework_path):
    # Pull updates if the framework is already cloned
    try:
        subprocess.run(["git", "-C", framework_path, "pull"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while pulling updates from the framework: {e}")
else:
    # Clone the framework if it is not already cloned
    try:
        subprocess.run(["git", "clone", "git@github.com:Phenomenology-group-uniandes/Uniandes_Framework.git"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while cloning the framework: {e}")
from Uniandes_Framework.delphes_reader.lhereader import LHE_Loader, readLHEF, get_event_by_child
from Uniandes_Framework.delphes_reader.root_analysis import get_kinematics_row, make_histograms, overlap_histos, Quiet

hint: Pulling without specifying how to reconcile divergent branches is
hint: discouraged. You can squelch this message by running one of the following
hint: commands sometime before your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.


Already up to date.
Welcome to JupyROOT 6.22/06


In [3]:
case = "woRHC"
# case = "wRHC"

limit = 'zp_upper_limit'
# limit = 'zp_lower_limit'

In [4]:
sim_path = os.path.join(os.sep, 'disco4', 'personal_folders', 'Cristian', 'SIMULATIONS', 'lq_zp', case, limit)

signals = ["zp_tau_tau", 
           "non-res_tau_tau",
          ]

M_U= [1000]
G_U = [1.75]
G_ZP = [1.5]

csv_sim_path = pd.read_csv(os.path.join("Uniandes_Framework", "SimulationsPaths.csv"))

#xs_path = { signal: os.path.join( os.path.dirname(os.getcwd()), "01_signal_production", f"xs_signals_{case}", signal, "XS_Matrix.csv") for signal in signals}

df = pd.concat(
    [csv_sim_path]
    +
    [ pd.DataFrame.from_dict(
    {
        "name": f"{case}_{limit}_" + signal + f"_mu_{m}_gu{gu}_gzp_{gzp}".replace('.', '_'), 
        "path": os.path.join(sim_path, signal,f'mu_{m}_gu{gu}_gzp_{gzp}'.replace('.', '_')),  
        'xs(pb)': 1.0
    }, 
    orient = "index").T for signal, m , gu, gzp in product(signals,M_U,G_U, G_ZP) ]
)

# save the new csv
csv_sim_path = os.path.join(os.getcwd(), "SimulationsPaths.csv")
df.to_csv(csv_sim_path, index = False)

In [5]:
def get_kinematics(run):
    run_name = os.path.basename(os.path.dirname(run))
    childs=readLHEF(run)
    if os.path.isfile(run):
        print('Reading', run_name)
    else:
        pass
    
    results = list()
    for child in childs:
        event = get_event_by_child(child)
        taus = event.getParticlesByIDs([-15,15])
        if len(taus) == 2:
            taus[0].SetName("#tau_{1}")
            taus[1].SetName("#tau_{2}")
            sum_taus = taus[0].TLV + taus[1].TLV
        else :
            continue
        row = get_kinematics_row(taus)
        row["m_ll(GeV)"] = sum_taus.M()
        row["sT(GeV)"] = row["pT_{#tau_{1}}(GeV)"] + row["pT_{#tau_{2}}(GeV)"]
        
        results.append(row)
    print(run_name, 'done!')
    return results


In [6]:
# Load the simulation

def load_simulation(x):
    signal, m, gu, gzp = x
    signal_key = f"{case}_{limit}_" + signal + f"_mu_{m}_gu{gu}_gzp_{gzp}".replace('.', '_')
    
    lhe_loader = LHE_Loader(signal_key, csv_sim_path)
    results = list()
    
    n_cores = 3
    with mp.Pool(n_cores) as pool:
        for result in list(pool.map(get_kinematics, lhe_loader.Forest)):
            results += result
    return signal_key, pd.DataFrame.from_records(results)

In [7]:
a = dict(map(load_simulation, product(signals,M_U,G_U, G_ZP)))

woRHC_zp_upper_limit_zp_tau_tau_mu_1000_gu1_75_gzp_1_5 imported with 1 trees!
/disco4/personal_folders/Cristian/SIMULATIONS/lq_zp/woRHC/zp_upper_limit/zp_tau_tau/mu_1000_gu1_75_gzp_1_5
Reading run_01
run_01 done!
woRHC_zp_upper_limit_non-res_tau_tau_mu_1000_gu1_75_gzp_1_5 imported with 1 trees!
/disco4/personal_folders/Cristian/SIMULATIONS/lq_zp/woRHC/zp_upper_limit/non-res_tau_tau/mu_1000_gu1_75_gzp_1_5
Reading run_01
run_01 done!


In [16]:
a.keys()

dict_keys(['woRHC_zp_upper_limit_zp_tau_tau_mu_1000_gu1_75_gzp_1_5', 'woRHC_zp_upper_limit_non-res_tau_tau_mu_1000_gu1_75_gzp_1_5'])

In [17]:
a['woRHC_zp_upper_limit_zp_tau_tau_mu_1000_gu1_75_gzp_1_5'].head()

,pT_{#tau_{1}}(GeV),#eta_{#tau_{1}},#phi_{#tau_{1}},Energy_{#tau_{1}}(GeV),Mass_{#tau_{1}}(GeV)),#Delta{R}_{#tau_{1}#tau_{2}},#Delta{#eta}_{#tau_{1}#tau_{2}},#Delta{#phi}_{#tau_{1}#tau_{2}},#Delta{pT}_{#tau_{1}#tau_{2}}(GeV),#Delta{#vec{pT}}_{#tau_{1}#tau_{2}}(GeV),#Delta{#vec{p}}_{#tau_{1}#tau_{2}}(GeV),pT_{#tau_{2}}(GeV),#eta_{#tau_{2}},#phi_{#tau_{2}},Energy_{#tau_{2}}(GeV),Mass_{#tau_{2}}(GeV)),m_ll(GeV),sT(GeV)
0,737.435706,0.052358,-1.592176,738.446738,-0.002607,3.568591,-1.692701,3.141593,0.0,1474.871412,2491.727447,737.435706,1.745059,1.549417,2175.757946,0.006954,2035.397297,1474.871412
1,963.546335,-0.471734,-1.892153,1072.759554,0.009102,3.187455,-0.538760,3.141593,0.0,1927.092671,2000.303342,963.546335,0.067027,1.249439,965.711547,-0.001828,1997.436903,1927.092671
2,134.514509,1.646428,1.013641,361.921514,-0.001573,4.102262,2.637982,3.141593,0.0,269.029019,561.039217,134.514509,-0.991553,-2.127952,206.238855,-0.001058,539.006413,269.029019
3,218.208173,0.169974,1.438902,221.367900,0.001240,3.204230,0.630467,3.141593,0.0,436.416346,458.733609,218.208173,-0.460493,-1.702690,241.755885,0.000101,458.280322,436.416346
4,361.081637,0.492887,-0.546463,405.836876,-0.002306,3.144525,0.135772,3.141593,0.0,722.163275,724.146768,361.081637,0.357114,2.595129,384.351838,0.001368,723.827973,722.163275


In [18]:
default_hist_bins_dict={
    "#Delta{R}":[96,0,7],
    "#Delta{#eta}":[80,-5,5],
    "#Delta{#phi}":[52,-3.25,3.25],
    "#Delta{pT}":[120, 0.0, 1500.0],
    "#Delta{#vec{pT}}":[240, 0.0, 4800.0],
    "#Delta{#vec{p}}":[240, 0.0, 4800.0],
    "MET(GeV)":[80, 0.0, 1000.0],
    "pT_": [160, 0.0, 2000.0],
    "sT(GeV)": [200, 0.0, 4000.0],
    "mT(GeV)": [200, 0.0, 4000.0],
    "#eta_":[80, -5, 5],
    "#phi_":[128, -3.2, 3.2],
    "Energy_":[80, 0.0, 1000.0],
    "m_ll(GeV)" : [160, 0.0, 2000.0]
}

In [28]:
root_histos = {'wRHC': {'zp_upper_limit': {}, 'zp_lower_limit': {}},
               'woRHC': {'zp_upper_limit': {}, 'zp_lower_limit': {}}}

for key in a.keys():
    Betard = key.split('_')[0]
    ZP_Limit = 'zp_' + key.split('_')[2] + '_limit'
    m_gu_gz = "mu" + key.split('mu')[1]
    root_histos[Betard][ZP_Limit][m_gu_gz] = {}

for key in a.keys():
    Betard = key.split('_')[0]
    ZP_Limit = 'zp_' + key.split('_')[2] + '_limit'
    m_gu_gz = "mu" + key.split('mu')[1]
    signal = ''.join(key.split('_')[4: len(key.split('_')) - 7]).replace('tau', ' #tau').replace('mu', ' mu')
    
    root_histos[Betard][ZP_Limit][m_gu_gz][signal] = make_histograms(a[key],hist_bins_dict = default_hist_bins_dict)

zp #tau #tau
non-res #tau #tau


In [29]:
root_histos

{'wRHC': {'zp_upper_limit': {}, 'zp_lower_limit': {}},
 'woRHC': {'zp_upper_limit': {'mu_1000_gu1_75_gzp_1_5': {'zp #tau #tau': {'pT_{#tau_{1}}(GeV)': <cppyy.gbl.TH1F object at 0x564b4aeeb8a0>,
     '#eta_{#tau_{1}}': <cppyy.gbl.TH1F object at 0x564b4aeeafa0>,
     '#phi_{#tau_{1}}': <cppyy.gbl.TH1F object at 0x564b4aeeab40>,
     'Energy_{#tau_{1}}(GeV)': <cppyy.gbl.TH1F object at 0x564b4aeea230>,
     '#Delta{R}_{#tau_{1}#tau_{2}}': <cppyy.gbl.TH1F object at 0x564b4aee99b0>,
     '#Delta{#eta}_{#tau_{1}#tau_{2}}': <cppyy.gbl.TH1F object at 0x564b4aee9400>,
     '#Delta{#phi}_{#tau_{1}#tau_{2}}': <cppyy.gbl.TH1F object at 0x564b4aee8af0>,
     '#Delta{pT}_{#tau_{1}#tau_{2}}(GeV)': <cppyy.gbl.TH1F object at 0x564b4aeef690>,
     '#Delta{#vec{pT}}_{#tau_{1}#tau_{2}}(GeV)': <cppyy.gbl.TH1F object at 0x564b4aeebc90>,
     '#Delta{#vec{p}}_{#tau_{1}#tau_{2}}(GeV)': <cppyy.gbl.TH1F object at 0x564b4aef04c0>,
     'pT_{#tau_{2}}(GeV)': <cppyy.gbl.TH1F object at 0x564b4aef8cb0>,
     '#eta_{#

In [30]:
# !rm -rf Most_Important_Kinematic_Histograms/

In [31]:
Important_features = ['m_ll(GeV)', 'sT(GeV)','#Delta{R}_{#tau_{1}#tau_{2}}', '#Delta{#phi}_{#tau_{1}#tau_{2}}', '#phi_{#tau_{1}}', '#eta_{#tau_{1}}', '#phi_{#tau_{2}}', 
                      '#Delta{pT}_{#tau_{1}#tau_{2}}(GeV)', '#Delta{#vec{pT}}_{#tau_{1}#tau_{2}}(GeV)', 'pT_{#tau_{1}}(GeV)', 'pT_{#tau_{2}}(GeV)']

folder = 'Most_Important_Kinematic_Histograms'
os.makedirs(folder, exist_ok=True)

for Betard in root_histos.keys():
    for ZP_Limit in root_histos[Betard].keys():
        for m_gu_gz in root_histos[Betard][ZP_Limit].keys():
            os.makedirs(os.path.join(folder, Betard, ZP_Limit, m_gu_gz), exist_ok=True)
            
            for feature in Important_features:
                histos, canvas, legend = overlap_histos(feature, root_histos[Betard][ZP_Limit][m_gu_gz], log_scale =True)
                with Quiet(): canvas.SaveAs(os.path.join(folder, Betard, ZP_Limit, m_gu_gz, f"{feature}.pdf".replace('{', '').replace('}', '')))